In [1]:
import os
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataver2/__results__.html
/kaggle/input/dataver2/data_ver2.h5
/kaggle/input/dataver2/__notebook__.ipynb
/kaggle/input/dataver2/__output__.json
/kaggle/input/dataver2/custom.css
/kaggle/input/dataver3/dataver3.h5
/kaggle/input/dataver3/__results__.html
/kaggle/input/dataver3/__notebook__.ipynb
/kaggle/input/dataver3/__output__.json
/kaggle/input/dataver3/custom.css
/kaggle/input/combine-data/combine_data.h5


In [3]:
# Thiết lập để hiển thị tối đa số cột (None có nghĩa là không giới hạn)
pd.set_option('display.max_columns', None)

# Thiết lập để hiển thị toàn bộ chiều rộng cột (tùy chọn)
pd.set_option('display.width', None)

# Direct Evalutation

## Completeness

* Độ đầy đủ: Dữ liệu phải chứa tất cả các thông tin cần thiết, không được thiếu các yếu tố quan trọng.
   
* Ý tưởng đo lường: Được tính bằng tỷ lệ giữa số lượng giá trị hợp lệ (không bị thiếu) và tổng số giá 
trị kỳ vọng.

### Completeness theo object

In [4]:
# combine_data.drop(columns=['row_completeness'], inplace=True)

In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

# Hàm kiểm tra giá trị thiếu (cho cả scalar và list)
def is_missing_value(x):
    missing_vals = [None, "", " ", "NULL", "null", "N/A", np.nan, "none", "nan"]
    if isinstance(x, list):
        return len(x) == 0 or any(item in missing_vals or pd.isna(item) for item in x)
    return x in missing_vals or pd.isna(x)

# Tính completeness cho một hàng
def compute_row_completeness(row):
    total = len(row)
    non_missing = sum(not is_missing_value(val) for val in row)
    return (non_missing / total) * 100 if total > 0 else 0

# Hàm chính để thêm cột completeness + in ra trung bình toàn bộ
def calc_completeness(df, new_col_name="row_completeness", round_ndigits=2, verbose=True):
    df = df.copy()  # tránh thay đổi gốc
    df[new_col_name] = df.progress_apply(
        lambda row: round(compute_row_completeness(row), round_ndigits), axis=1
    )

    if verbose:
        completeness_overall = df[new_col_name].mean()
        print(f"✅ Giá trị completeness trung bình của dataset là: {completeness_overall:.2f}%")
    
    return df

### Completeness toàn bộ dataset

In [6]:
# dataset_ver1 = calc_completeness(combine_data)

## Consistency

* Tính nhất quán: Dữ liệu phải đồng nhất giữa các nguồn và hệ thống khác nhau, không có xung đột 
hoặc trùng lặp. 
* Ý tưởng đo lường: Được tính bằng tỷ lệ giữa số lượng điều kiện hợp lệ và tổng số điều kiện cần kiểm tra. 

Consistency theo Object: Là tỷ lệ giữa số lượng điều kiện hợp lệ trong object và tổng số điều kiện cần kiểm tra. 

Các loại kiểm tra tính hợp lệ cơ bản:
* Miền giá trị (Domain Range): Giá trị nằm trong một khoảng nhất định
* Dữ liệu không rỗng (Non-null): Không được để trống
* Loại dữ liệu (Data Type): Phải thuộc kiểu dữ liệu nhất định
* Ràng buộc logic (Logical Constraints): Giá trị phải thỏa mãn một điều kiện logic
* Tính duy nhất (Uniqueness): Giá trị không trùng lặp trong tập dữ liệu
* Tính khóa ngoại (Foreign Key Integrity): Giá trị phải tồn tại trong một danh sách hợp lệ. 

#### Miền giá trị (Domain Range): 
* Các giá trị trong cột user_gender phải thuộc tập {0, 1, 2}.
* user_id và course_id phải đúng định dạng string, user_id bắt đầu bằng "U_", course_id bắt đầu bằng "C_".
* enroll_time của user với course phải nằm trong khoảng thời gian thu thập dữ liệu [2019; 2021].
* total_course_comments, user_course_num_comment, user_course_num_replies là giá trị kiểu số nguyên int không được là số âm tức >= 0.
* num_problem, num_correct_problem, video_duration, actual_watch_time và total_watch_time là giá trị kiểu list, mỗi phần tử trong list phải >= 0: tức không được là số âm.

#### Dữ liệu không rỗng (Non-null): 
* Các cột quan trọng: user_id, course_id, school_id, teacher_id, course_filed không được null
  

#### Loại dữ liệu (Data Type): 
* Cột enroll_time thuộc kiểu Unix time stamp để thuận tiện trong việc xử lí và sử dụng sau này, kiểu Unix time stamp được biểu diễn trong python dưới dạng int.
* Các cột: user_gender, total_course_comments, user_course_num_comment, user_course_num_replies thuộc kiểu int.
* Các cột:  user_id, course_id, course_name, course_prerequisites, course_about thuộc kiểu string.
* Các cột: school_id, teacher_id, video_id, ccid, watch_id, exercise_id, doing_exercise_id, concept_id, course_field, school_name, school_about, school_motto,teacher_name, teacher_about, teacher_job_title, teacher_org_name, video_duration, total_watch_time, actual_watch_time, num_correct_problem, concept_ccid, concept_name, concept_field, concept_prerequisite thuộc kiểu dữ liệu object thuộc kiểu list string.

#### Ràng buộc logic (Logical Constraints):
* actual_watch_time <= total_watch_time: Không thể xem nhiều hơn tổng thời gian đã xem.
* actual_watch_time <= video_duration: Không thể xem nhiều hơn thời lượng video.
* user_course_num_comment + user_course_num_replies <= total_course_comments: Tổng số bình luận và phản hồi của người dùng cho khóa học <= Tổng số tương tác của khóa học đó.
* Mỗi concept_id không được là phần tử trong chính concept_prerequisite: Một khái niệm không thể là tiên quyết của chính nó.
* num_correct_problem <= num_problem: Không thể làm đúng nhiều hơn tổng số lượng câu hỏi của bài tập.

#### Tính duy nhất (Uniqueness):
* user_id + course_id phải là duy nhất: Mỗi người dùng chỉ học một khóa học một lần, Một user không thể đăng ký 2 lần cùng một khóa học nếu mỗi dòng là một phiên học
* Đồng thời kiểm tra 1 object có bị trùng lặp với các object khác hay không (xét trường hợp đầy đủ thuộc tính và trường hợp bỏ thuộc tính chính: user_id và course_id)?

#### Tính khóa ngoại (Foreign Key Integrity): Giá trị trong các cột id phải tồn tại trong các file của MOOCCubeX entities.

| Cột trong dataset chính       | Phải tồn tại trong file          | Ghi chú                                                             |
|-------------------------------|----------------------------------|----------------------------------------------------------------------|
| `user_id`                     | `user.json`                      | Mỗi dòng là một người học                                           |
| `course_id`                   | `course.json`                    | Mỗi dòng là một khóa học                                            |
|                                 |
| `school_id`                   | `school.json`                    | Trường cung cấp khóa học                                            |
| `teacher_id`                  | `teacher.json`                   | Giảng viên dạy khóa học                                             |
| `concept_id`                  | `concept.json`                   | Concept được gán cho khóa học                                       |
| `exercise_id`                 | `exercise-problem.json`          | Tập hợp bài tập trong khóa học                                      |
| `ccid`        | `video_id-ccid.txt`              | Map giữa video và ccid                                  |

In [7]:
import time
import polars as pl

def pl_read_file(file_path):
    file_extension = os.path.splitext(file_path)[1]
    if file_extension == '.json':
        return pl.read_ndjson(file_path)
    elif file_extension == '.csv':
        return pl.read_csv(file_path)
    elif file_extension == '.txt':
        return pl.read_csv(file_path,separator='\t',has_header=False)
    else:
        raise ValueError(f"Unsupported file extension: {file_extension}")

In [8]:
# user = pl_read_file("/kaggle/input/mooccubex-entities/user.json")
# course = pl_read_file("/kaggle/input/mooccubex-entities/course.json")
# school = pl_read_file("/kaggle/input/mooccubex-entities/school.json")
# teacher = pl_read_file("/kaggle/input/mooccubex-entities/teacher.json")
# concept = pl_read_file("/kaggle/input/mooccubex-entities/concept.json")
# exercise_problem = pl_read_file("/kaggle/input/mooccubex-relations/exercise-problem.txt")
# video_ccid = pl_read_file("/kaggle/input/mooccubex-relations/video_id-ccid.txt")

In [9]:
# # Process video_ccid: drop column_1 and rename column_2 to ccid
# video_ccid = video_ccid.drop("column_1").rename({"column_2": "ccid"})

# # Process exercise_problem: drop column_2 and rename column_1 to exercise_id
# exercise_problem = exercise_problem.drop("column_2").rename({"column_1": "id"})

# # Chỉ giữ lại cột 'id' cho từng DataFrame
# user = user[['id']]
# teacher = teacher[['id']]
# course = course[['id']]
# school = school[['id']]
# concept = concept[['id']]

In [10]:
# # chuyển sang pandas
# user = user.to_pandas()
# teacher = teacher.to_pandas()
# course = course.to_pandas()
# school = school.to_pandas()
# concept = concept.to_pandas()
# exercise_problem = exercise_problem.to_pandas()
# video_ccid = video_ccid.to_pandas()

In [11]:
# combine_data.drop(columns=['row_completeness'], inplace=True)

In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed
import ast

# ======================
# UTILITY FUNCTIONS
# ======================

def is_missing_value(value):
    """Kiểm tra giá trị thiếu cho cả scalar và danh sách.
    
    Args:
        value: Giá trị cần kiểm tra (có thể là scalar, danh sách, hoặc bất kỳ kiểu dữ liệu nào).
    
    Returns:
        bool: True nếu giá trị được coi là thiếu, False nếu không.
    """
    
    # Danh sách các giá trị được coi là thiếu
    missing_vals = {None, "", " ", "NULL", "null", "N/A", "none", "nan"}
    
    # Trường hợp giá trị là danh sách hoặc tuple
    if isinstance(value, (list, tuple)):
        if len(value) == 0:  # Danh sách rỗng được coi là thiếu
            return True
        # Chỉ coi là thiếu nếu tất cả phần tử đều thiếu
        return all(is_missing_value(item) for item in value)
    
    # Trường hợp giá trị là chuỗi
    if isinstance(value, str):
        # Loại bỏ khoảng trắng ở hai đầu và kiểm tra
        cleaned_value = value.strip()
        return cleaned_value in missing_vals or cleaned_value == ""
    
    # Trường hợp giá trị là số hoặc các kiểu khác
    return value in missing_vals or pd.isna(value)

# ======================
# NON-NULL CHECK
# ======================

def check_non_null_vectorized(df):
    """Kiểm tra non-null cho các cột quan trọng: user_id, course_id, school_id, teacher_id, course_filed.
    Nếu có bất kỳ 1 cột nào bị null thì object đó sẽ không đạt."""
    results = pd.Series(np.ones(len(df)), index=df.index)
    errors = pd.Series([None] * len(df), index=df.index)
    
    # Các cột quan trọng cần kiểm tra non-null
    required_columns = ['user_id', 'course_id', 'school_id', 'teacher_id', 'course_field']
    
    # Tìm các cột tồn tại trong DataFrame
    existing_columns = [col for col in required_columns if col in df.columns]
    
    if not existing_columns:
        print("Cảnh báo: Không tìm thấy cột nào cần kiểm tra non-null trong DataFrame")
        return results, errors
    
    # Khởi tạo danh sách để lưu các cột bị thiếu cho từng dòng
    missing_columns_per_row = []
    
    for idx in df.index:
        missing_cols = []
        has_missing = False
        
        for col in existing_columns:
            if is_missing_value(df.loc[idx, col]):
                missing_cols.append(col)
                has_missing = True
        
        if has_missing:
            results[idx] = 0
            errors[idx] = f"Các cột bị thiếu: {', '.join(missing_cols)}"
        
        missing_columns_per_row.append(missing_cols)
    
    return results, errors

def process_batch(batch_df):
    """Process a batch of records for non-null check only."""
    # Initialize result and error series
    batch_results = {
        'non_null': pd.Series(np.ones(len(batch_df)), index=batch_df.index)
    }
    
    batch_errors = {
        'non_null': pd.Series([None] * len(batch_df), index=batch_df.index)
    }
    
    # Run non-null check
    nn_results, nn_errors = check_non_null_vectorized(batch_df)
    
    # Update batch results and errors
    batch_results['non_null'] = nn_results
    batch_errors['non_null'] = nn_errors
    
    return batch_results, batch_errors

# ======================
# MAIN FUNCTION
# ======================

def evaluate_consistency_improved(df, user=None, teacher=None, course=None, school=None, concept=None, exercise_problem=None, video_ccid=None, sample_size=None):
    """Đánh giá non-null consistency với phương pháp cải tiến."""
    print("Bắt đầu kiểm tra non-null consistency...")
    
    # Lấy mẫu nếu cần
    if sample_size and len(df) > sample_size:
        print(f"Lấy mẫu ngẫu nhiên {sample_size} dòng từ {len(df)} dòng...")
        df_sample = df.sample(sample_size, random_state=42)
    else:
        df_sample = df
    
    print(f"Số lượng dòng cần kiểm tra: {len(df_sample)}")
    
    # Xác định kích thước batch phù hợp
    cpu_count = multiprocessing.cpu_count()
    batch_size = max(100, min(1000, len(df_sample) // (cpu_count * 2)))
    
    # Chia thành các batch
    batches = [df_sample.iloc[i:i+batch_size] for i in range(0, len(df_sample), batch_size)]
    print(f"Chia dữ liệu thành {len(batches)} batch, mỗi batch khoảng {batch_size} dòng")
    
    # Khởi tạo kết quả chung
    results = {
        'non_null': pd.Series(np.ones(len(df_sample)), index=df_sample.index)
    }
    
    errors = {
        'non_null': pd.Series([None] * len(df_sample), index=df_sample.index)
    }
    
    # Xử lý từng batch
    print(f"Bắt đầu xử lý {len(batches)} batch...")
    
    for i, batch in enumerate(tqdm(batches, desc="Xử lý các batch")):
        batch_results, batch_errors = process_batch(batch)
        
        # Cập nhật kết quả
        for check_type in results.keys():
            results[check_type].loc[batch.index] = batch_results[check_type]
            errors[check_type].loc[batch.index] = batch_errors[check_type]
    
    # Tính điểm consistency (chỉ dựa trên non_null)
    consistency_scores = results['non_null'] * 100
    
    # Tạo DataFrame kết quả
    results_df = pd.DataFrame({
        'user_id': df_sample['user_id'] if 'user_id' in df_sample.columns else None,
        'course_id': df_sample['course_id'] if 'course_id' in df_sample.columns else None,
    })
    
    # Thêm kết quả kiểm tra non_null
    results_df['non_null'] = results['non_null']
    
    # Thêm điểm consistency
    results_df['consistency_score'] = consistency_scores
    
    # Chuyển đổi errors thành DataFrame
    error_df = pd.DataFrame(errors)
    
    return results_df, error_df

def show_missing_rate_by_column(df):
    """Hiển thị tỉ lệ missing của từng cột được yêu cầu kiểm tra."""
    print("\nTỈ LỆ MISSING CỦA TỪNG CỘT:")
    print("-" * 40)
    
    # Các cột quan trọng cần kiểm tra
    required_columns = ['user_id', 'course_id', 'school_id', 'teacher_id', 'course_filed']
    
    # Tìm các cột tồn tại trong DataFrame
    existing_columns = [col for col in required_columns if col in df.columns]
    
    if not existing_columns:
        print("Không tìm thấy cột nào cần kiểm tra trong DataFrame")
        return
    
    missing_stats = {}
    
    for col in existing_columns:
        # Đếm số lượng missing values
        missing_count = df[col].apply(is_missing_value).sum()
        total_count = len(df)
        missing_rate = (missing_count / total_count) * 100
        
        missing_stats[col] = {
            'missing_count': missing_count,
            'total_count': total_count,
            'missing_rate': missing_rate
        }
        
        print(f"{col:<15}: {missing_count:>6}/{total_count:<6} ({missing_rate:>6.2f}%)")
    
    # Hiển thị tổng quan
    print("-" * 40)
    total_missing = sum(stats['missing_count'] for stats in missing_stats.values())
    total_cells = sum(stats['total_count'] for stats in missing_stats.values())
    overall_missing_rate = (total_missing / total_cells) * 100
    
    print(f"{'Tổng cộng':<15}: {total_missing:>6}/{total_cells:<6} ({overall_missing_rate:>6.2f}%)")
    
    return missing_stats

def summarize_consistency(results_df, original_df=None):
    """Tóm tắt kết quả kiểm tra non-null consistency."""
    print("\nTÓM TẮT KẾT QUẢ NON-NULL CONSISTENCY:")
    print(f"Tổng số dòng đã kiểm tra: {len(results_df)}")
    print(f"Consistency trung bình: {results_df['consistency_score'].mean():.2f}%")
    
    # Thống kê theo loại kiểm tra
    pass_rate = results_df['non_null'].mean() * 100
    print(f"Tỷ lệ đạt non_null: {pass_rate:.2f}%")
    
    # Thống kê điểm consistency
    score_ranges = [(0, 50), (50, 80), (80, 100), (100, 100)]
    for low, high in score_ranges:
        if low == high:
            count = (results_df['consistency_score'] == low).sum()
            percent = count / len(results_df) * 100
            print(f"Số dòng đạt điểm {low}: {count} ({percent:.2f}%)")
        else:
            count = ((results_df['consistency_score'] >= low) & (results_df['consistency_score'] < high)).sum()
            percent = count / len(results_df) * 100
            print(f"Số dòng đạt điểm {low}-{high}: {count} ({percent:.2f}%)")
    
    # Hiển thị tỉ lệ missing của từng cột nếu có dữ liệu gốc
    if original_df is not None:
        show_missing_rate_by_column(original_df)

def run_consistency_check(df, user=None, teacher=None, course=None, school=None, concept=None, exercise_problem=None, video_ccid=None, completeness_overall=None, sample_size=None):
    """Hàm chính để chạy kiểm tra non-null consistency."""
    import time
    
    start_time = time.time()
    
    # Hiển thị tỉ lệ missing của từng cột trước khi kiểm tra
    print("PHÂN TÍCH DỮ LIỆU TRƯỚC KHI KIỂM TRA:")
    original_df = df.sample(sample_size, random_state=42) if sample_size and len(df) > sample_size else df
    show_missing_rate_by_column(original_df)
    
    # Chạy đánh giá consistency
    results_df, error_df = evaluate_consistency_improved(
        df, user, teacher, course, school, concept, exercise_problem, video_ccid, sample_size
    )
    
    # Tóm tắt kết quả (không cần truyền original_df vì đã hiển thị ở trên)
    summarize_consistency(results_df)
    
    # Báo cáo thời gian
    execution_time = time.time() - start_time
    print(f"\nThời gian thực thi: {execution_time:.2f} giây")
    
    if sample_size and len(df) > sample_size:
        # Ước tính thời gian cho toàn bộ dataset
        estimated_full_time = execution_time * (len(df) / sample_size)
        hours = estimated_full_time // 3600
        minutes = (estimated_full_time % 3600) // 60
        seconds = estimated_full_time % 60
        
        print(f"Ước tính thời gian cho toàn bộ dataset ({len(df)} dòng): "
              f"{int(hours)} giờ {int(minutes)} phút {int(seconds)} giây")
    
    return results_df, error_df

# ======================

In [13]:
# results_df, error_df = run_consistency_check(combine_data, user, teacher, course, school, concept, exercise_problem, video_ccid, sample_size=10000)

In [14]:
# results_df, error_df = run_consistency_check(combine_data, user, teacher, course, school, concept, exercise_problem, video_ccid)

In [15]:
# print(error_df['non_null'].head(10))

# Dataset version 1: Raw dataset

In [16]:
combine_data = pd.read_hdf('/kaggle/input/combine-data/combine_data.h5', key='df')

## Completeness

In [17]:
# v1 = calc_completeness(combine_data)

## Consistency

In [18]:
results_df, error_df = run_consistency_check(
    combine_data, 
    sample_size=None  # hoặc None để chạy toàn bộ
)

PHÂN TÍCH DỮ LIỆU TRƯỚC KHI KIỂM TRA:

TỈ LỆ MISSING CỦA TỪNG CỘT:
----------------------------------------
user_id        :      0/7284500 (  0.00%)
course_id      :      0/7284500 (  0.00%)
school_id      :  26541/7284500 (  0.36%)
teacher_id     :   9960/7284500 (  0.14%)
----------------------------------------
Tổng cộng      :  36501/29138000 (  0.13%)
Bắt đầu kiểm tra non-null consistency...
Số lượng dòng cần kiểm tra: 7284500
Chia dữ liệu thành 7285 batch, mỗi batch khoảng 1000 dòng
Bắt đầu xử lý 7285 batch...


Xử lý các batch: 100%|██████████| 7285/7285 [08:05<00:00, 14.99it/s]



TÓM TẮT KẾT QUẢ NON-NULL CONSISTENCY:
Tổng số dòng đã kiểm tra: 7284500
Consistency trung bình: 26.68%
Tỷ lệ đạt non_null: 26.68%
Số dòng đạt điểm 0-50: 5340654 (73.32%)
Số dòng đạt điểm 50-80: 0 (0.00%)
Số dòng đạt điểm 80-100: 0 (0.00%)
Số dòng đạt điểm 100: 1943846 (26.68%)

Thời gian thực thi: 518.36 giây


In [19]:
combine_data.isnull().mean()

user_id                    0.000000
course_id                  0.000000
enroll_time                0.000000
course_name                0.000000
course_prerequisites       0.000000
course_about               0.000000
course_field               0.731191
school_id                  0.003643
school_name                0.003643
school_about               0.003643
school_motto               0.003643
teacher_id                 0.001367
teacher_name               0.001367
teacher_about              0.001367
teacher_job_title          0.001367
teacher_org_name           0.001367
video_id                   0.379174
video_ccid                 0.379174
video_duration             0.379174
user_gender                0.000000
watch_id                   0.999437
total_watch_time           0.999437
actual_watch_time          0.999437
exercise_id                0.019379
exercise_num_problem       0.019379
doing_exercise_id          0.996887
doing_num_correct          0.996887
concept_id                 0

# Dataset version 2: Loại bỏ các cột null trên ngưỡng threshold kết hợp listwises deletion

In [20]:
# v2 = pd.read_hdf('/kaggle/input/dataver2/data_ver2.h5', key='df')

In [21]:
# display(v2)

## Completeness

In [22]:
# v2 = calc_completeness(v2)

## Consistency

In [23]:
# results_df, error_df = run_consistency_check(v2, user, teacher, course, school, concept, exercise_problem, video_ccid)

In [24]:
# error_df['non_null'].unique()

In [25]:
# # Bung toàn bộ phần tử trong các list thành từng dòng
# exploded_items = v2['teacher_name'].explode()

# # Tổng số phần tử sau khi explode
# total_count = len(exploded_items)

# # Số phần tử bị thiếu
# null_count = exploded_items.isna().sum()

# # Tính tỷ lệ phần trăm
# null_percent = (null_count / total_count) * 100 if total_count > 0 else 0

# # In kết quả
# print(f"Tổng số phần tử: {total_count}")
# print(f"Số phần tử thiếu (NaN): {null_count}")
# print(f"Tỷ lệ thiếu: {null_percent:.2f}%")

In [26]:
# # Tập hợp tất cả các phần tử từ các danh sách trong cột
# all_items = v2['teacher_name'].explode()

# # Lấy các giá trị duy nhất và in ra
# unique_items = all_items.unique()

# # In từng giá trị
# for item in unique_items:
#     print(item)

# Dataset version 3: Điền khuyết bằng giá trị mode

In [27]:
# v3 = pd.read_hdf('/kaggle/input/dataver3/dataver3.h5', key='df')

In [28]:
# display(v3)

In [29]:
# v3.info()

## Completeness

In [30]:
# v3 = calc_completeness(v3)

In [31]:
# completeness_overall = v3['row_completeness'].mean()

In [32]:
# v3['row_completeness']

## Consistency

In [33]:
# results_df, error_df = run_consistency_check(v3, user, teacher, course, school, concept, exercise_problem, video_ccid, completeness_overall, sample_size=1000)

In [34]:
# error_df['logic'].unique()

In [35]:
# for val in v3['course_about'].unique():
#     if isinstance(val, str) and len(val) < 5:
#         print(val)


# Dataset version 4: Điền khuyết bằng phương pháp GNN

In [36]:
# v4 = pd.read_hdf('', key='df')

In [37]:
# display(v4)

## Completeness

In [38]:
# v4 = calc_completeness(v4)

## Consistency

In [39]:
# results_df, error_df = run_consistency_check(v4, user, teacher, course, school, concept, exercise_problem, video_ccid)

In [40]:
# %whos